# Get backend information with Qiskit

This page explains how to use Qiskit to find information about your available backends.

## List backends

To view the backends you have access to, you can either view a list on the [Compute resources page,](https://quantum-computing.ibm.com/services/resources?tab=yours) or you can use the [`QiskitRuntimeService.backends()`](../api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#backends) method. This method returns a list of [`IBMBackend`](../api/qiskit-ibm-runtime/qiskit_ibm_runtime.IBMBackend) instances.

In [1]:
# Initialize your account 
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService()

service.backends()

[<IBMBackend('ibmq_qasm_simulator')>,
 <IBMBackend('simulator_mps')>,
 <IBMBackend('simulator_statevector')>,
 <IBMBackend('ibm_lagos')>,
 <IBMBackend('ibm_perth')>,
 <IBMBackend('ibm_brisbane')>,
 <IBMBackend('simulator_extended_stabilizer')>,
 <IBMBackend('simulator_stabilizer')>,
 <IBMBackend('ibm_nairobi')>]

The [`QiskitRuntimeService.backend()`](../api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#backend) method (note that this is singular: *backend*) takes the name of the backend as the input parameter and returns an [`IBMBackend`](../api/qiskit-ibm-runtime/qiskit_ibm_runtime.IBMBackend) instance representing that particular backend:

In [2]:
service.backend("ibmq_qasm_simulator")

<IBMBackend('ibmq_qasm_simulator')>

## Filter backends

You can also filter the available backends by their properties. For more general filters, you can make advanced functions using a lambda function. Refer to the [API documentation](../api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#backends) for more details.

Let’s try getting only backends that fit these criteria:

*   Are real quantum devices (`simulator=False`)
*   Are currently operational (`operational=True`)
*   Have at least 5 qubits (`min_num_qubits=5`)

In [3]:
service.backends(simulator=False, operational=True, min_num_qubits=5)

[<IBMBackend('ibm_lagos')>,
 <IBMBackend('ibm_perth')>,
 <IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_nairobi')>]

A similar method is [`QiskitRuntimeService.least_busy()`](../api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#least_busy), which takes the same filters as `backends()` but returns the backend that matches the filters and has the least number of jobs pending in the queue:

In [4]:
service.least_busy(operational=True, min_num_qubits=5)

<IBMBackend('ibmq_qasm_simulator')>

## Static backend information

Some information about a backend does not change regularly, such as its name, version, the number of qubits it has, and the types of features it supports. This information is available as attributes of the `backend` object.

The following cell builds a description of a backend.

In [5]:
backend = service.backend("ibm_lagos")

print(
    f"Name: {backend.name}\n"
    f"Version: {backend.version}\n"
    f"No. of qubits: {backend.num_qubits}\n"
)

Name: ibm_lagos
Version: 2
No. of qubits: 7



For a full list of attributes, see the [`IBMBackend` API documentation](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.IBMBackend).

## Dynamic backend information

Backends can also have properties that change whenever the backed is calibrated, such as qubit frequency and operation error rates. Backends are usually calibrated every 24 hours, and their properties update after the calibration sequence completes. These properties can be used when optimizing quantum circuits or to construct noise models for a classical simulator.


### Qubit properties

The `backend.qubit_properties` method returns information about the qubits' physical attributes. This includes the qubit frequency in GHz and decay times (`t1` and `t2`) in µs.

In [6]:
backend.qubit_properties(0)  # properties of qubit 0

IBMQubitProperties(t1=0.00010568006375765179, t2=3.753491927659795e-05, frequency=5235355398.037262, anharmonicity=-339867138.55915606)

### Instruction properties

The `backend.target` attribute is a `qiskit.transpiler.Target` object: an object that contains all the information needed to transpile a circuit for that backend. This includes instruction errors and durations. For example, the following cell gets the properties for a `cx` gate acting between qubits 0 and 1.

In [7]:
backend.target["cx"][(0,1)]

InstructionProperties(duration=5.76e-07, error=0.010379675306311759, calibration=Schedule cx)

The following cell shows the properties for a measurement operation (including the readout error) on qubit 0.

In [8]:
backend.target["measure"][(0,)]

InstructionProperties(duration=7.893333333333333e-07, error=0.011199999999999988, calibration=Schedule measure)